# Getting started

Before we get started, ensure you have the required packages installed:

```sh
pip install reductionml scikit-learn
```

In [45]:
from reductionml import Workspace, FormatType


First, we need to define the configuration we are going to use. You can think of this as the definition of the structure of the reduction stack, including all things that will affect learning and prediction. Things such as hyper parameters.

We will create an instance of [`Coin`](https://jackgerrits.com/reductionml/book/coin.html) as the only reduction in our stack which provides solves for linear regression problems.

In [46]:
config = {
    "entryReduction": {
        "config": {},
        "typename": "Coin"
    }, 
    "globalConfig": {
        "interactions": [
            ["Default", "Default"]
        ]
    }
}

workspace = Workspace.create_from_config(config)

We're going to use the diabetes dataset from sklearn.

In [47]:
import sklearn.datasets
import sklearn.model_selection
dataset = sklearn.datasets.load_diabetes()
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(dataset.data, dataset.target)


In [48]:
def format_input(features, label):

    return {
        "label": {
            "value": label,
            "weight": 1.0
        },
        "features": {
            ":default": features.tolist()
        }
    }

parser = workspace.create_parser(FormatType.Json)
for pass_num in range(10):
    for (input_features, input_label) in zip(X_train, y_train):
        features, label = parser.parse(format_input(input_features, input_label))
        prediction = workspace.learn(features, label)
    
    mse_error = 0
    for (input_features, input_label) in zip(X_test, y_test):
        features, label = parser.parse(format_input(input_features, input_label))
        prediction = workspace.predict(features)
        mse_error += (label.value - prediction.prediction) ** 2

    print(f"MSE error at pass {pass_num}: {mse_error / len(X_test)}")


MSE error at pass 0: 8021.6635847471625
MSE error at pass 1: 3825.003633391618
MSE error at pass 2: 3636.0612630902106
MSE error at pass 3: 3508.012612874407
MSE error at pass 4: 3437.157331053816
MSE error at pass 5: 3397.1723666073763
MSE error at pass 6: 3368.089415636847
MSE error at pass 7: 3344.064184543594
MSE error at pass 8: 3322.720712088957
MSE error at pass 9: 3302.6458895202413
